In [1]:
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

/home/vco/Projects/pm4py-dcr/notebooks
/home/vco/Projects/pm4py-dcr


In [2]:
import pm4py
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.discovery.dcr_discover import algorithm as alg

# File names

In [3]:
event_log_file = 'datasets/sepsis_cases_event_log.xes'
dcrxml_file = 'models/sepsis_dcr.xml'

# Mine DCR graph using the DisCoveR miner

In [4]:
event_log = pm4py.read_xes(event_log_file,return_legacy_log_object=True)

/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 2026.70it/s]


In [5]:
traces = event_log

In [6]:
dcr_discover = alg.Discover()
dcr_graph = dcr_discover.mine(log=event_log,findAdditionalConditions=True) # this will need updating to the new pm4py standards (for now it works)
print(dcr_graph)

{'events': {'IV Liquid', 'Admission NC', 'ER Registration', 'Admission IC', 'Release B', 'Leucocytes', 'ER Sepsis Triage', 'Release E', 'IV Antibiotics', 'ER Triage', 'LacticAcid', 'Return ER', 'CRP', 'Release A', 'Release C', 'Release D'}, 'conditionsFor': {'IV Liquid': set(), 'Admission NC': {'ER Registration', 'ER Triage'}, 'ER Registration': set(), 'Admission IC': {'ER Registration', 'ER Triage'}, 'Release B': {'ER Triage', 'Leucocytes', 'CRP', 'ER Sepsis Triage', 'ER Registration'}, 'Leucocytes': set(), 'ER Sepsis Triage': set(), 'Release E': {'Leucocytes', 'CRP', 'Admission NC', 'ER Sepsis Triage'}, 'IV Antibiotics': {'ER Sepsis Triage'}, 'ER Triage': set(), 'LacticAcid': set(), 'Return ER': {'Admission IC', 'Release E', 'Release A', 'Release C', 'Release D'}, 'CRP': set(), 'Release A': {'ER Registration', 'ER Triage'}, 'Release C': {'Leucocytes', 'CRP', 'Admission NC', 'ER Sepsis Triage'}, 'Release D': {'Leucocytes', 'CRP', 'Admission NC', 'ER Sepsis Triage'}}, 'milestonesFor': 

/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


# Write the graph to file

In [7]:
dcr_exporter.export_dcr_xml(dcr=dcr_graph,output_file_name=dcrxml_file,dcr_title="Sepsis Cases DCR")

# Read the dcrxml graph file (works also with graphs downlaoded from the dcrgraphs.net portal

In [8]:
dcr_graph_loaded = dcr_importer.apply(dcrxml_file)
print(dcr_graph_loaded)

{'events': {'ReleaseE', 'ReleaseB', 'ReleaseA', 'LacticAcid', 'IVAntibiotics', 'AdmissionIC', 'AdmissionNC', 'ERTriage', 'ReleaseD', 'ERRegistration', 'ReleaseC', 'Leucocytes', 'CRP', 'ERSepsisTriage', 'IVLiquid', 'ReturnER'}, 'labels': {'ReleaseE', 'ReleaseB', 'ReleaseA', 'LacticAcid', 'IVAntibiotics', 'AdmissionIC', 'AdmissionNC', 'ERTriage', 'ReleaseD', 'ERRegistration', 'ReleaseC', 'Leucocytes', 'CRP', 'ERSepsisTriage', 'IVLiquid', 'ReturnER'}, 'labelMapping': set(), 'conditionsFor': {'AdmissionNC': {'ERRegistration', 'ERTriage'}, 'AdmissionIC': {'ERRegistration', 'ERTriage'}, 'ReleaseB': {'ERTriage', 'ERRegistration', 'Leucocytes', 'CRP', 'ERSepsisTriage'}, 'ReleaseE': {'AdmissionNC', 'Leucocytes', 'CRP', 'ERSepsisTriage'}, 'IVAntibiotics': {'ERSepsisTriage'}, 'ReturnER': {'ReleaseE', 'ReleaseA', 'AdmissionIC', 'ReleaseD', 'ReleaseC'}, 'ReleaseA': {'ERRegistration', 'ERTriage'}, 'ReleaseC': {'AdmissionNC', 'Leucocytes', 'CRP', 'ERSepsisTriage'}, 'ReleaseD': {'AdmissionNC', 'Leucoc

# Play with the execution semantics (note milestones and no-responses are not supported yet)

In [9]:
dcr = dcr_graph
print(dcr['events']) # all the events

{'IV Liquid', 'Admission NC', 'ER Registration', 'Admission IC', 'Release B', 'Leucocytes', 'ER Sepsis Triage', 'Release E', 'IV Antibiotics', 'ER Triage', 'LacticAcid', 'Return ER', 'CRP', 'Release A', 'Release C', 'Release D'}


In [10]:
print(dcr_semantics.enabled(dcr)) # check enabled events

{'IV Liquid', 'ER Triage', 'LacticAcid', 'ER Registration', 'Leucocytes', 'CRP', 'ER Sepsis Triage'}


In [13]:
event_to_execute = 'ER Registration' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

[!] Event ER Registration not enabled!
{'IV Liquid', 'ER Triage', 'LacticAcid', 'Leucocytes', 'CRP', 'ER Sepsis Triage'}


In [12]:
dcr_semantics.is_accepting(dcr) # this checks the acceptance criteria for the dcr graph

True